In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
%matplotlib inline
from sklearn import preprocessing
from array import *
from sklearn.datasets import load_iris

In [2]:
data = pd.read_csv("D:\college\sem5\Soft computing\iris.csv", header=None)
decision_attribute =  data[4]
data.drop(data.columns[4], inplace = True, axis = 1)
feature=data.columns
feature

Int64Index([0, 1, 2, 3], dtype='int64')

In [3]:
data

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [4]:
columns = data.to_numpy() 
min_max_scaler= preprocessing.MinMaxScaler()
columns_scaled = min_max_scaler.fit_transform(columns)
normalized = pd.DataFrame(columns_scaled)

In [5]:
graph = normalized.corr()

In [6]:
graph.shape

(4, 4)

In [7]:
graph

,0,1,2,3
0,1.000000,-0.109369,0.871754,0.817954
1,-0.109369,1.000000,-0.420516,-0.356544
2,0.871754,-0.420516,1.000000,0.962757
3,0.817954,-0.356544,0.962757,1.000000


In [8]:
graph = graph.to_numpy()
for i in range(len(graph)):
    graph[i][i] = 0

#  predetermined threshold 

In [9]:
s = np.sum(graph)
threshold = s / ((graph.shape[0] ** 2)-len(graph))
print(threshold)

0.2943392419598983


# Remove all the edges with weights less than a predetermined threshold.

In [10]:
for i in range(len(graph)):
    for j in range(len(graph[0])):
        if graph[i][j] < threshold :
            graph[i][j] = 0

In [11]:
graph

array([[0.        , 0.        , 0.87175416, 0.81795363],
       [0.        , 0.        , 0.        , 0.        ],
       [0.87175416, 0.        , 0.        , 0.9627571 ],
       [0.81795363, 0.        , 0.9627571 , 0.        ]])

# Compute the importance of a node as the sum of weights of all edges incident


In [12]:
def node_to_delete(matrix):
    nodes = np.argmax(np.sum(matrix, axis = 0).reshape(matrix.shape[0], 1))
    print("Deleted node : ", nodes)
    return nodes

def delete_edges_of_node(node_number, matrix):
    for i in range(len(matrix)):
        matrix[node_number][i] = 0
        matrix[i][node_number] = 0

    return matrix

# Select the most important node and remove it from the graph.

In [13]:
arr=array('i',[])
nodes_deleted = []
while np.sum(graph) != 0: 
    nodes = node_to_delete(graph)
    for i in range(1):
        arr.append(nodes)    
    
    nodes_deleted.append(nodes)
    graph = delete_edges_of_node(nodes, graph)
    

Deleted node :  2
Deleted node :  0


In [14]:
graph

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [15]:
print(arr)

array('i', [2, 0])


In [16]:
data.iloc[:,[0,2]]

,0,2
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4
...,...,...
145,6.7,5.2
146,6.3,5.0
147,6.5,5.2
148,6.2,5.4


In [17]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

# A = set of attributes
def equivalence_class_util(x, A):
    val = []
    for j in A:
        val.append(x[j])
    return val


# A = set of attributes
def equivalence_class(data, A):
    eq_class = []
    for i, x in enumerate(data.values):
        val = equivalence_class_util(x, A)
    
    found_class = False
    for c in eq_class:
        if(val == equivalence_class_util(data.values[c[0]], A)):
            found_class = True
            c.append(i)
            break
    
    if(not found_class):
        eq_class.append([i])
    return eq_class


# P = set of attributes
# X = set of objects
def len_lower_approximation(data, P, X):
    len_lower_P = 0
    EC = equivalence_class(data, P)
    for x in X:
        for c in EC:
            if((x in c) and (intersection(c, X) == c)):
                len_lower_P += 1
                break
    return len_lower_P;


# P = a subset of attributes
# eq_class_decision = equivalence class of decision attribute
def gamma(data, P, eq_class_Q):
    if(not P):
        return 0
    denom = len(data)
    numer = 0
    
    for qi in eq_class_Q:
        numer += len_lower_approximation(data, P, qi)
        
    return numer / denom


# data should not contain decision attribute column
# features should not contain decision attribute
def QRGA(data, features, decision_attribute):
    reduct = []
    T = []
    
    num_features = len(features)

    eq_class_decision = []
    
    for i, x in enumerate(data.values):
        found_class = False
    for c in eq_class_decision:
        if(decision_attribute[i] == decision_attribute[c[0]]):
            found_class = True
            c.append(i)
            break
    
    if(not found_class):
        eq_class_decision.append([i])
        
    is_cond = True
    while(is_cond):
        T = reduct.copy()
        for x in range(num_features):
            if(x in reduct):
                continue
            tmp = reduct.copy()
            tmp.append(x)
            if(gamma(data, tmp, eq_class_decision) > gamma(data, T, eq_class_decision)):
                T.append(x)
                reduct = T.copy()
        is_cond = gamma(data, reduct, eq_class_decision) > gamma(data, features, eq_class_decision)
    reduct.sort()
    return reduct

In [18]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['target'] = iris.target


In [19]:
from sklearn import preprocessing

arr = df.to_numpy()
min_max_scaler= preprocessing.MinMaxScaler()
scaled_arr = min_max_scaler.fit_transform(arr)
data_norm = pd.DataFrame(scaled_arr)
data_norm.drop(data_norm.columns[0], inplace = True, axis = 1)
wine = data_norm.to_numpy()

In [20]:
##decision = df.loc[ :,['target']]
##decision

In [21]:
num_features = len(data.values[0])
features = [i for i in range(num_features)]
reduced_features = QRGA(data, features, decision_attribute)

In [22]:
print(reduced_features)

[0]
